<h2>Load all data we need from all datasets. Get all the columns and things we need. Save cleaned dfs to use to make d3 vis

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import plotly.express as px
import geopandas as gpd
import requests


<h3>Load Data

In [4]:
API_TOKEN = 'tRPy6aES2cCa26Y6QSvwPzC8eswqiiVFPGAJ'
BASE_URL = "https://api.iucnredlist.org/api/v4"

In [6]:
def get_species_by_group(class_name):
    """
    Fetch all species for a taxonomic group (e.g., 'Amphibia')
    Returns a list of species dictionaries
    """
    url = f"{BASE_URL}/taxa/class/{class_name}"
    headers = {
        "Authorization": API_TOKEN,
        "Accept": "application/json"
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Error:", response.status_code)
        return []
    
    data = response.json()
    return data.get('assessments', [])

def get_threats(sis_id):
    url = f"{BASE_URL}/threats/species/{sis_id}"
    headers = {"Authorization": API_TOKEN, "Accept": "application/json"}
    response = requests.get(url, headers=headers)
    return response.json().get("threats", [])


def get_assessments(assessment_id):
    url = f"{BASE_URL}/assessment/{assessment_id}"
    headers = {"Authorization": API_TOKEN, "Accept": "application/json"}
    response = requests.get(url, headers=headers)
    return response.json()



In [7]:
#amphibians
amphibians = get_species_by_group("Amphibia")
print(f"Fetched {len(amphibians)} amphibian species")
print(amphibians[:2])  # preview first 2

#reptiles
reptiles = get_species_by_group("Reptilia")
print(f"Fetched {len(reptiles)} reptile species")
print(reptiles[:2])  # preview first 2

Fetched 100 amphibian species
[{'year_published': '2020', 'latest': True, 'possibly_extinct': False, 'possibly_extinct_in_the_wild': False, 'sis_taxon_id': 12776, 'url': 'https://www.iucnredlist.org/species/12776/508612', 'taxon_scientific_name': 'Mantella aurantiaca', 'red_list_category_code': 'EN', 'assessment_id': 508612, 'scopes': [{'description': {'en': 'Global'}, 'code': '1'}]}, {'year_published': '2016', 'latest': False, 'possibly_extinct': False, 'possibly_extinct_in_the_wild': False, 'sis_taxon_id': 1301, 'url': 'https://www.iucnredlist.org/species/1301/511335', 'taxon_scientific_name': 'Anhydrophryne rattrayi', 'red_list_category_code': 'VU', 'assessment_id': 511335, 'scopes': [{'description': {'en': 'Global'}, 'code': '1'}]}]
Fetched 100 reptile species
[{'year_published': '2021', 'latest': True, 'possibly_extinct': False, 'possibly_extinct_in_the_wild': False, 'sis_taxon_id': 10041, 'url': 'https://www.iucnredlist.org/species/10041/495907', 'taxon_scientific_name': 'Heosemy

In [8]:
def create_df(animal_json):
    rows = []
    for species in animal_json:
        assessment = get_assessments(species["assessment_id"])
        
        if isinstance(assessment, dict) and "result" in assessment:
            assessment = assessment["result"][0]

        # Safely extract threats and threat codes
        threats_list = assessment.get("threats", []) or []
        threats = [
            (t.get("description", {}).get("en") if isinstance(t.get("description"), dict)
             else t.get("description"))
            for t in threats_list
        ]
        threats_code = [
            (t.get("code", {}) if len(threats_list)>=0
             else 0)
            for t in threats_list
        ]

        # Safely extract habitats
        habitat_list = assessment.get("habitats", []) or []
        habitats = [
            (h.get("description", {}).get("en") if isinstance(h.get("description"), dict)
             else h.get("description"))
            for h in habitat_list
        ]

        # Safely extract population trend
        pop = assessment.get("population_trend", {})
        if isinstance(pop, dict):
            population_trend = (
                pop.get("description", {}).get("en")
                if isinstance(pop.get("description"), dict)
                else pop.get("description")
            )
        
        
        rows.append({
            "taxon":assessment['taxon']['class_name'],
            "species_name": species["taxon_scientific_name"],
            "red_list_category": species["red_list_category_code"],
            "threats": threats,
            "threats_code":threats_code,
            "population_trend": population_trend,
            "habitats": habitats
        })
    return pd.DataFrame(rows)

In [9]:
amphibian_df = create_df(amphibians)

amphibian_df.columns
amphibian_df.shape
amphibian_df.head(3)

,taxon,species_name,red_list_category,threats,threats_code,population_trend,habitats
0,AMPHIBIA,Mantella aurantiaca,EN,"[Intentional use (species is the target), Mini...","[5_1_1, 3_2, 2_1_1, 7_1_3, 5_3_3, 2_1_2, 1_1, ...",Decreasing,"[Forest - Subtropical/Tropical Swamp, Wetlands..."
1,AMPHIBIA,Anhydrophryne rattrayi,VU,"[Agro-industry plantations, Unspecified specie...","[2_2_2, 8_1_1, 7_1_1, 5_3_3]",Unknown,"[Forest - Temperate, Grassland - Temperate]"
2,AMPHIBIA,Bokermannohyla itapoty,LC,[],[],Stable,[Grassland - Subtropical/Tropical High Altitud...


In [10]:
reptile_df = create_df(reptiles)

reptile_df.columns
reptile_df.shape
reptile_df.head(3)

,taxon,species_name,red_list_category,threats,threats_code,population_trend,habitats
0,REPTILIA,Heosemys annandalii,CR,[Intentional use: (subsistence/small scale) [h...,"[5_4_1, 1_1, 4_1, 9_3_3, 11_1, 5_1_1]",Decreasing,[Wetlands (inland) - Seasonal/Intermittent Fre...
1,REPTILIA,Iguanognathus werneri,DD,[],[],Decreasing,[Unknown]
2,REPTILIA,Indotestudo elongata,CR,"[Scale Unknown/Unrecorded, Unintentional effec...","[2_1_4, 5_2_2, 5_1_1, 2_1_2, 4_1, 9_3_2, 5_1_2]",Decreasing,"[Shrubland - Subtropical/Tropical Dry, Forest ..."


In [11]:
combined_df = pd.concat([amphibian_df, reptile_df]).reset_index(drop=True)

combined_df.shape
combined_df.head()

,taxon,species_name,red_list_category,threats,threats_code,population_trend,habitats
0,AMPHIBIA,Mantella aurantiaca,EN,"[Intentional use (species is the target), Mini...","[5_1_1, 3_2, 2_1_1, 7_1_3, 5_3_3, 2_1_2, 1_1, ...",Decreasing,"[Forest - Subtropical/Tropical Swamp, Wetlands..."
1,AMPHIBIA,Anhydrophryne rattrayi,VU,"[Agro-industry plantations, Unspecified specie...","[2_2_2, 8_1_1, 7_1_1, 5_3_3]",Unknown,"[Forest - Temperate, Grassland - Temperate]"
2,AMPHIBIA,Bokermannohyla itapoty,LC,[],[],Stable,[Grassland - Subtropical/Tropical High Altitud...
3,AMPHIBIA,Pseudis platensis,LC,"[Agro-industry farming, Small-holder grazing, ...","[2_1_3, 2_3_2, 2_3_3, 7_1_1, 1_1, 4_1]",Stable,[Wetlands (inland) - Seasonal/Intermittent Fre...
4,AMPHIBIA,Tsingymantis antitra,EN,"[Small-holder grazing, ranching or farming, In...","[2_3_2, 5_3_1, 3_2]",Unknown,[Wetlands (inland) - Permanent Rivers/Streams/...


In [12]:
risk_map = {
    "LC": 0,   # Least Concern
    "NT": 1,   # Near Threatened
    "VU": 2,   # Vulnerable
    "EN": 3,   # Endangered
    "CR": 4,   # Critically Endangered
    "EW": 5,   # Extinct in the Wild
    "EX": 6    # Extinct
}

combined_df["risk_score"] = combined_df["red_list_category"].map(risk_map)



In [13]:
def threats_map(threat_codes):
    threats = {
        '1':"Residential & commercial development",
        '2':"Agriculture & aquaculture",
        '3':"Energy production & mining",
        '4':"Transportation & service corridors",
        '5':"Biological resource use",
        '6':"Human intrusions & disturbance",
        '7':"Natural system modifications",
        '8':"Invasive and other problematic species, genes & diseases",
        '9':"Pollution",
        '10':"Geological events",
        '11':"Climate change & severe weather",
        '12':"Other options"
    }
    output = []
    for code in threat_codes:
        val = code[0]
        output.append(threats[val])
    return output

In [15]:
combined_df['threats'] = combined_df['threats_code'].apply(threats_map)
combined_df.head()

,taxon,species_name,red_list_category,threats,threats_code,population_trend,habitats,risk_score
0,AMPHIBIA,Mantella aurantiaca,EN,"[Biological resource use, Energy production & ...","[5_1_1, 3_2, 2_1_1, 7_1_3, 5_3_3, 2_1_2, 1_1, ...",Decreasing,"[Forest - Subtropical/Tropical Swamp, Wetlands...",3.0
1,AMPHIBIA,Anhydrophryne rattrayi,VU,"[Agriculture & aquaculture, Invasive and other...","[2_2_2, 8_1_1, 7_1_1, 5_3_3]",Unknown,"[Forest - Temperate, Grassland - Temperate]",2.0
2,AMPHIBIA,Bokermannohyla itapoty,LC,[],[],Stable,[Grassland - Subtropical/Tropical High Altitud...,0.0
3,AMPHIBIA,Pseudis platensis,LC,"[Agriculture & aquaculture, Agriculture & aqua...","[2_1_3, 2_3_2, 2_3_3, 7_1_1, 1_1, 4_1]",Stable,[Wetlands (inland) - Seasonal/Intermittent Fre...,0.0
4,AMPHIBIA,Tsingymantis antitra,EN,"[Agriculture & aquaculture, Biological resourc...","[2_3_2, 5_3_1, 3_2]",Unknown,[Wetlands (inland) - Permanent Rivers/Streams/...,3.0


<h1> Sliders: Threat slider → applicable when threat list contains: </h1>

| Slider	 |       Threat Codes to look for	   |    Effect |
| :--------- | :-----------------------------: | ---------: |
| Temperature rise|	11	   |       Added risk for extinction |
| Habitat loss	|    1, 5, 6, 7, 10	| Added risk for extinction|
| CO₂ concentration	| 2, 3, 4, 9	  |  Added risk for extinction |


<h1>For Co2 slider:</h1>

<h3>Use emissions df to see which industries cause which greenhouse gases to go up and then we can affect those species.
</h3>

<h3>
For example if we see that agriculture increases the amount of methane in the air, when the slider for methane is increased, the species that have agriculture related things as a threat should have their risk scores increased.

In [84]:
emissions_df = pd.read_csv('indicator_quarterly.csv')

emissions_df.head(3)

,ObjectId,Country,ISO2,ISO3,Indicator,Unit,Source,CTS Code,CTS Name,CTS Full Descriptor,...,2022Q4,2023Q1,2023Q2,2023Q3,2023Q4,2024Q1,2024Q2,2024Q3,2024Q4,2025Q1
0,1,Advanced Economies,NaN,AETMP,Quarterly greenhouse gas (GHG) air emissions a...,Million metric tons of CO2 equivalent,Organisation for Economic Co-operation and Dev...,ECNGA,Greenhouse Gas Emissions (GHG); Air Emissions ...,"Environment, Climate Change, Greenhouse Gas Em...",...,64.688709,61.319185,65.867074,68.262567,66.039285,62.452184,73.597480,75.162258,71.925035,70.263853
1,2,Advanced Economies,NaN,AETMP,Quarterly greenhouse gas (GHG) air emissions a...,Million metric tons of CO2 equivalent,Organisation for Economic Co-operation and Dev...,ECNGA,Greenhouse Gas Emissions (GHG); Air Emissions ...,"Environment, Climate Change, Greenhouse Gas Em...",...,64.104215,64.183297,65.752729,65.954354,65.597731,65.374440,72.340776,72.275717,72.779925,74.291176
2,3,Advanced Economies,NaN,AETMP,Quarterly greenhouse gas (GHG) air emissions a...,Million metric tons of CO2 equivalent,Organisation for Economic Co-operation and Dev...,ECNGA,Greenhouse Gas Emissions (GHG); Air Emissions ...,"Environment, Climate Change, Greenhouse Gas Em...",...,0.248269,0.237558,0.236841,0.235717,0.241366,0.233355,0.236395,0.238220,0.242984,0.235360


<h1>For Temperature Slider</h1>

<h3>
For example if the user increases the temperature slider, we should increase the risk factor for species that have a threat code thats affected by temperature. We can also have different multipliers for certain threat codes. For example if a specie has a threat code 11 meaning its threatened by climate change and severe weather, then its affected a lot by an increase in temp.

In [ ]:
# Assigns sensitivites to temp. so if temp increases by 2 celsius, you multiply that by the sensitivity of a species and 
# see the new risk factor. the risk factor will be a number 0-6. (see risk_map function above). We probably need to 
# update the sensitivities so that it doesn't display every species as extinct as soon as the user sets a temp increase 
# of 6 but this is the gist
def assignSensitivity(threat_codes):
    sens = {
        '7':0.5,
        '9':0.3,
        '10':0.7, #increase temp causes more severe geological events
        '11':1,
    }
    output = []
    for threat in threat_codes:
        if threat in sens.keys():
            output.append(sens[threat])
        else:
            output.append(0)
    return output



In [17]:
#temp =  baseline temp of 1.2 degrees Celsius (global temp) + slider value (user input temp)
#risk_increase = (new_temp - baseline) * sensitivity


temp = 1.2 #+ slider_val
risk_increase = (temp-1.2) #* sensitivity

<h1>For Habitat Loss Slider</h1>
<h3>We can have a slider from like 0-100% and simulate it using the same idea as temps. Like using a sensitivity score for each specie.</h3>
<h3>
For example if the user increases the habitat loss slider, we should increase the risk factor for species that have a threat code thats affected by habitat loss. We can also have different multipliers for certain threat codes. For example if a specie has a threat code of 1, 5, 6, 7, 10 meaning its threatened by habitat loss, then its affected a lot by an increase in loss.